In [ ]:
%pip install kaggle --upgrade split-folders

In [6]:
from azureml.core import Workspace, Dataset, Experiment

ws = Workspace.from_config()

In [2]:
# Export Kaggle configuration variables
%env KAGGLE_USERNAME=szelor
%env KAGGLE_KEY=21cd02df9ad71ca909694a3733b5b22d

env: KAGGLE_USERNAME=szelor
env: KAGGLE_KEY=21cd02df9ad71ca909694a3733b5b22d


In [3]:
# Download the Pneumonia dataset. ISSUE: Requires Python 3.6 AzureML kernel which is not available in newer Compute Instances
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia -p ./datafile

100%|███████████████████████████████████████| 2.29G/2.29G [00:19<00:00, 310MB/s]
100%|██████████████████████████████████████| 2.29G/2.29G [00:35<00:00, 69.9MB/s]
unzip:  cannot find or open ./chest-xray-pneumonia.zip, ./chest-xray-pneumonia.zip.zip or ./chest-xray-pneumonia.zip.ZIP.


In [5]:
!unzip -q ./datafile/chest-xray-pneumonia.zip -d ./files

^C


In [38]:
import json
import os
src = "./pneunomiafiles/"
train_validation_ratio = 20

# Retrieving default datastore that got automatically created when we setup a workspace
workspaceblobstore = ws.get_default_datastore().name

# Path to the training and validation files
train_annotations_file = os.path.join(src, "train_annotations.jsonl")
validation_annotations_file = os.path.join(src, "validation_annotations.jsonl")

# sample json line dictionary
json_line_sample = {
    "image_url": "AmlDatastore://"
    + workspaceblobstore
    + "/"
    + os.path.basename(os.path.dirname(src)),
    "label": "",
}

index = 0
# Scan each sub directary and generate jsonl line
with open(train_annotations_file, "w") as train_f:
    with open(validation_annotations_file, "w") as validation_f:
        for className in os.listdir(src):
            subDir = src + className
            if not os.path.isdir(subDir):
                continue
            # Scan each sub directary
            print("Parsing " + subDir)
            for image in os.listdir(subDir):
                json_line = dict(json_line_sample)
                json_line["image_url"] += f"/{className}/{image}"
                json_line["label"] = className

                if index % train_validation_ratio == 0:
                    # validation annotation
                    validation_f.write(json.dumps(json_line) + "\n")
                else:
                    # train annotation
                    train_f.write(json.dumps(json_line) + "\n")
                index += 1

Parsing ./pneunomiafiles/NORMAL
Parsing ./pneunomiafiles/PNEUMONIA


In [42]:
# Retrieving default datastore that got automatically created when we setup a workspace
ds = ws.get_default_datastore()
ds.upload(src_dir="./pneunomiafiles/", target_path="pneunomiafiles")

Uploading an estimated of 633 files
Target already exists. Skipping upload for pneunomiafiles/NORMAL/.amlignore
Target already exists. Skipping upload for pneunomiafiles/NORMAL/.amlignore.amltmp
Target already exists. Skipping upload for pneunomiafiles/NORMAL/._IM-0001-0001.jpeg
Target already exists. Skipping upload for pneunomiafiles/NORMAL/._IM-0003-0001.jpeg
Target already exists. Skipping upload for pneunomiafiles/NORMAL/._IM-0005-0001.jpeg
Target already exists. Skipping upload for pneunomiafiles/NORMAL/._IM-0006-0001.jpeg
Target already exists. Skipping upload for pneunomiafiles/NORMAL/._IM-0007-0001.jpeg
Target already exists. Skipping upload for pneunomiafiles/NORMAL/._IM-0009-0001.jpeg
Target already exists. Skipping upload for pneunomiafiles/NORMAL/._IM-0010-0001.jpeg
Target already exists. Skipping upload for pneunomiafiles/NORMAL/._IM-0011-0001-0001.jpeg
Target already exists. Skipping upload for pneunomiafiles/NORMAL/._IM-0011-0001-0002.jpeg
Target already exists. Skippin

$AZUREML_DATAREFERENCE_8732bedc68984aa7ac0193514fa9acf4

In [43]:
from azureml.core import Dataset
from azureml.data import DataType

# get existing training dataset
training_dataset_name = "pneumoniaTrainingDataset"
if training_dataset_name in ws.datasets:
    training_dataset = ws.datasets.get(training_dataset_name)
    print("Found the training dataset", training_dataset_name)
else:
    # create training dataset 
    training_dataset = Dataset.Tabular.from_json_lines_files(
        path=ds.path("pneunomiafiles/train_annotations.jsonl"),
        set_column_types={"image_url": DataType.to_stream(ds.workspace)},
    )
    training_dataset = training_dataset.register(
        workspace=ws, name=training_dataset_name
    )
# get existing validation dataset
validation_dataset_name = "pneumoniaValidationDataset"
if validation_dataset_name in ws.datasets:
    validation_dataset = ws.datasets.get(validation_dataset_name)
    print("Found the validation dataset", validation_dataset_name)
else:
    # create validation dataset
    validation_dataset = Dataset.Tabular.from_json_lines_files(
        path=ds.path("pneunomiafiles/validation_annotations.jsonl"),
        set_column_types={"image_url": DataType.to_stream(ds.workspace)},
    )
    validation_dataset = validation_dataset.register(
        workspace=ws, name=validation_dataset_name
    )
print("Training dataset name: " + training_dataset.name)
print("Validation dataset name: " + validation_dataset.name)

Training dataset name: pneumoniaTrainingDataset
Validation dataset name: pneumoniaValidationDataset


In [41]:
training_dataset.to_pandas_dataframe()

,image_url,label
0,workspaceblobstore/pneunomiafiles/NORMAL/.amli...,NORMAL
1,workspaceblobstore/pneunomiafiles/NORMAL/._IM-...,NORMAL
2,workspaceblobstore/pneunomiafiles/NORMAL/._IM-...,NORMAL
3,workspaceblobstore/pneunomiafiles/NORMAL/._IM-...,NORMAL
4,workspaceblobstore/pneunomiafiles/NORMAL/._IM-...,NORMAL
...,...,...
592,workspaceblobstore/pneunomiafiles/PNEUMONIA/._...,PNEUMONIA
593,workspaceblobstore/pneunomiafiles/PNEUMONIA/._...,PNEUMONIA
594,workspaceblobstore/pneunomiafiles/PNEUMONIA/._...,PNEUMONIA
595,workspaceblobstore/pneunomiafiles/PNEUMONIA/._...,PNEUMONIA
